In [1]:
import pandas as pd
import gspread

In [3]:

gc = gspread.service_account()

sh = gc.open("Example spreadsheet")

print(sh.sheet1.get('A1'))

MalformedError: Service account info was not in the expected format, missing fields client_email.